In [3]:
'''
Code for piping the majority and minority classes to their appropriate models, and running the models
'''
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime, timedelta
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#Other model will go here as well
import LACER as lc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sgcrf import SparseGaussianCRF
import pickle

#Features to use in our model, c = categorical, d = numeric
c = ['Anonymous','AssignTo', 'RequestType', 'RequestSource','CD','Direction', 'ActionTaken', 'APC' ,'AddressVerified']
d = ['Latitude', 'Longitude']
    
#Slightly editied lacer funcions
def preprocessing(df, start_date, end_date):
    """
    Filters dataframe by specified start and end_dates and runs CleanedFrame on it.  
    """ 
    #Filter dataframe by dates 
    df = df[(df['Just Date'] >= start_date) & (df['Just Date'] <= end_date)]
    df = lc.CleanedFrame(df)
    return df

def lacer(df, df1, train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num):
    """
    Trains 2 GCRF models on data from specified CD and Request Type which is assigned to fulfill request. 
    Uses specified start and end dates for training and testing to creat train and test sets. 
    """

    #Create Training and Testing Sets
    dftrain = preprocessing(df, train_start_date, train_end_date)
    dftrain = dftrain.reset_index(drop = True)
    dftest = preprocessing(df1, test_start_date, test_end_date)
    dftest = dftest.reset_index(drop = True)

    #Reserve test set for training on all 3 models.
    
    y_train, y_test = lc.CreateTestSet(dftest, predictor_num)
    y_test = y_test.reshape((-1, 1))
   

    ## 2 Models
    #Model1: CD
    modelCD = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=10000)
    dftrainCD = dftrain[dftrain['CD'] == CD].reset_index(drop = True)

    X_trainCD, X_testCD = lc.CreateTrainSet(dftrainCD, predictor_num)
    X_testCD = X_testCD.reshape((-1, 1))
    modelCD.fit(X_trainCD, X_testCD)

    y_predCD = modelCD.predict(y_train)

    #Model2: Request_type
    modelRT = SparseGaussianCRF(lamL=0.1, lamT=0.1, n_iter=10000)
    dftrainRT = dftrain[dftrain['RequestType'] == request_type].reset_index(drop = True)

    X_trainRT, X_testRT = lc.CreateTrainSet(dftrainRT, predictor_num)
    X_testRT = X_testRT.reshape((-1, 1))

    modelRT.fit(X_trainRT, X_testRT)

    y_predRT = modelRT.predict(y_train)


    #Average out all predictions
    y_predFinal = (y_predCD + y_predRT )/2

    # Return models
    return modelCD, modelRT

"""
Retu whether or not a number is greater than 11. 
"""
def gelev(val): 
    if val <= 11: 
        return 0
    else: 
        return 1

'''
Preprocessing function. Takes in the file path to the data and loads it in a DataFrame, then calcuates the elapsed days per request and marks them as more than or less than eleven days. Then it encodes the appropriate values and returns the train data, labels, and the formatted dataframe.
'''
def preprocess(df, formatted=False):
    if not formatted:
        df['Just Date'] = df['Just Date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
    df['Eleven'] = df['ElapsedDays'].apply(gelev, 0)
    #Put desired columns into dataframe, drop nulls. 
    dfn = df.filter(items = c + d + ['ElapsedDays','Just Date','CreatedDate','ClosedDate'])
    dfn = dfn.dropna()
    
    #Separate data into explanatory and response variables
    XCAT = dfn.filter(items = c).values
    XNUM = dfn.filter(items = d).values
    y = dfn['ElapsedDays'] <= 11
    
    #Encode cateogrical data and merge with numerical data
    onehotencoder = OneHotEncoder()
    #Dump encoder class data to pickle
    pickle.dump(onehotencoder, open("encoder.pickl", "wb"))
    XCAT = onehotencoder.fit_transform(XCAT).toarray()
    X = np.concatenate((XCAT, XNUM), axis=1)
    return X,y, dfn

'''
Runs the model that classifies each request as more than or less than/equal to 11 days. Parameters are the hyperparameters for the model itself, and the train data and labels.
'''
def estimation_model(estimators, depth,X,y):    
    rf = RandomForestClassifier(n_estimators = estimators, max_depth = depth)
    print('creating train, test, val split')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)
    #Train Model on 3 yrs data on
    print('training model')
    rf.fit(X_train, y_train)
    #Test model
    print('testing model')
    y_vpred = rf.predict(X_val)
    print('dump to pickle')
    pickle.dump(rf, open('randomForest.pkl', 'wb'))
    #Print Accuracy Function results
    print("Accuracy:",metrics.accuracy_score(y_val, y_vpred))
    print("Precision, Recall, F1Score:",metrics.precision_recall_fscore_support(y_val, y_vpred, average = 'binary'))
    return rf

'''
Will be replaced with model for requests over 11 days
'''
def minority_class_model(df):
    # this needs to return a model for the deployment to work -- Andy
    return 0,0

'''
Takes a file path to the data, runs the appropriate preprocessing steps, and uses the model to classify everything into the majority and minority class. Returns a dataframe with the majority class and a separate one with the minority class.
'''
def split_to_models(df,formatted=False):
    print('Calculating train data and labels')
    X, y, df = preprocess(df,formatted)
    print('Creating 11 day classifier')
    #Train on 3 yrs before data
    model_eleven = estimation_model(50,20,X,y)
    df['LessEqualEleven'] = model_eleven.predict(X)
    df['LessEqualEleven'] = df['LessEqualEleven'].apply(lambda x: int(x))
    df_sgcrf = df[df['LessEqualEleven'] == 1.0]
    df_other = df[df['LessEqualEleven'] == 0.0]
    return df_sgcrf, df_other

'''
Takes the data file path and parameters for the lacer model and runs the pipeline to get the appropriate dataframes, then runs both models. This is the main function you should run from this file.
'''
'''
def create_split_models(df,train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num):
    df_sgcrf, df_other = split_to_models(df)
    print('running SGCRF')
    #10 weeks
    modelCD, modelRT = lacer(df_sgcrf.copy(),df_sgcrf.copy(), train_start_date, train_end_date, test_start_date, test_end_date, request_type, CD, predictor_num)
    print('running (other model)')
    #a,b = minority_class_model(df_other)
    
    # send to pickle so that flask can access it
    print('dumping to pickle')
    pickle.dump(modelCD, open('modelCD.pkl','wb'))
    pickle.dump(modelRT, open('modelRT.pkl','wb'))
    #return (modelCD, modelRT), (a, b)
'''
#Demo code
def create_models(df,start_date, request_type, CD, predictor_num):
    start = datetime.strptime(start_date,'%Y-%m-%d')
    X, y, dfn = preprocess(df)
    #past three years
    df_three = dfn[(dfn['Just Date'] <= start-timedelta(weeks=11)) &
                   (dfn['Just Date'] >= start-timedelta(weeks=11) - timedelta(days=365*3))]
    df_sgcrf,ignore = split_to_models(df_three,True)
    train_end_date = df_sgcrf.iloc[-50]['Just Date']
    modelCD, modelRT = lacer(df_sgcrf.copy(),df_sgcrf.copy(), train_end_date - timedelta(weeks=10), train_end_date, "2017-06-02", "2017-09-02", request_type, CD, predictor_num)
    pickle.dump(modelCD, open('modelCD.pkl','wb'))
    pickle.dump(modelRT, open('modelRT.pkl','wb'))
    #return (modelCD, modelRT), (a, b)

'''
Beginning code to update the model after requests have been made.
'''
def update_model(requests,modelCD,modelRT,modelMin):
    df_request = pd.DataFrame(data=requests,columns=c+d)
    X, y, df = preprocess(df_request)
    df_sgcrf = df[df['ElapsedDays'] <= 11.0]
    df_other = df[df['ElapsedDays'] > 11.0]
    modelCD.fit(np.asarray(df_sgcrf['CD']),np.asarray(df_sgcrf['ElapsedDays']))
    modelRT.fit(np.asarray(df_sgcrf['RequestType']),np.asarray(df_sgcrf['ElapsedDays']))
    #modelMin.fit will go here
    #dump back to pickle - or we can just return the models themselves
    pickle.dump(modelCD, open('modelCD.pkl'), 'wb')
    pickle.dump(modelRT, open('modelRT.pkl'), 'wb')

In [11]:
create_models(pd.read_csv('fservice.csv'),'2018-01-01','Graffiti Removal',5,50)

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Calculating train data and labels


C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Creating 11 day classifier
creating train, test, val split
training model
testing model
dump to pickle
Accuracy: 0.9657530375812376
Precision, Recall, F1Score: (0.9719114610841696, 0.9930240087911225, 0.9823543114493147, None)


C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ElapsedHours'] = df.apply(lambda x: elapsedHours(x['CreatedDate'],x['ClosedDate']),axis=1)
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ClosedDate'] = df['ClosedDate'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
C:\Users\ngeta\p-requests\311_Requests_Model\LACER.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [6]:
X,y, dfn = preprocess(pd.read_csv('fservice.csv'))

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
dfn.head()

,Anonymous,AssignTo,RequestType,RequestSource,CD,Direction,ActionTaken,APC,AddressVerified,Latitude,Longitude,ElapsedDays,Just Date,CreatedDate,ClosedDate
1,Y,CCAC,Graffiti Removal,Mobile App,13.0,N,SR Created,East Los Angeles APC,Y,34.084714,-118.260479,11,2017-01-01,01/01/2017 12:45:00 AM,01/12/2017 05:10:00 PM
3,N,WVA,Graffiti Removal,Mobile App,12.0,W,SR Created,North Valley APC,Y,34.256993,-118.604730,3,2017-01-01,01/01/2017 01:01:00 AM,01/04/2017 05:33:00 PM
4,N,WVA,Graffiti Removal,Mobile App,12.0,W,SR Created,North Valley APC,Y,34.257061,-118.594999,3,2017-01-01,01/01/2017 01:09:00 AM,01/04/2017 05:34:00 PM
5,Y,NEGB_ELA,Graffiti Removal,Mobile App,1.0,E,SR Created,East Los Angeles APC,Y,34.090852,-118.226214,2,2017-01-01,01/01/2017 01:16:00 AM,01/03/2017 06:55:00 PM
6,N,NC,Bulky Items,Mobile App,4.0,N,SR Created,Central APC,Y,34.084822,-118.333307,6,2017-01-01,01/01/2017 01:21:00 AM,01/07/2017 09:46:00 AM


In [10]:
dfn.columns

Index(['Anonymous', 'AssignTo', 'RequestType', 'RequestSource', 'CD',
       'Direction', 'ActionTaken', 'APC', 'AddressVerified', 'Latitude',
       'Longitude', 'ElapsedDays', 'Just Date', 'CreatedDate', 'ClosedDate'],
      dtype='object')